In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
img_fl = np.load('../../downloading_data/images.npz')
ids = img_fl['ids']

In [ ]:
def mag_to_flx(mag):
    """
    Input: an AB magnitude
    Output: a flux in microjansky (muJy)
    Works with np arrays and pd Series
    """
    return 10**(29-(48.60/2.5)) * 10**(-mag/2.5)

def read_lephare_spectrum_file(id):
    """
    Reads the file containing the output spectra for the inputted ID
    """
    with open(f'output_spectra/Id{str(id)[-9:]}.spec','r') as f:
        specfile_lines = f.read().split('\n')
    specfile_lines = [row.split() for row in specfile_lines]
    return specfile_lines

def get_lephare_spectrum(id, i): # i from 0 to 2
    """
    Inputs: An ID, and an integer in [0,1,2]
    Outputs: wavelengths, and a spectrum in muJy. The spectrum is the 
        LePHARE best fit spectrum for:
            0: Galaxy
            1: Quasar
            2: Star
    """
    file_lines = read_lephare_spectrum_file(id)
    spectra = np.array(file_lines[171:][:-1]).astype('float32')
    restart_indices = np.where(np.diff(spectra, axis=0)[:,0]<0)[0]
    spec_list = np.split(spectra, restart_indices+1, axis=0)
    x,y_mag = spec_list[i][:,0],spec_list[i][:,1]
    x,y_mag = x[(y_mag<100) & (y_mag!=0)], y_mag[(y_mag<100)&(y_mag!=0)] # Clipping ceilinged SED values
    y = mag_to_flx(y_mag)
    return x,y

Now the various wavelength and flux arrays generated by the above function are not of the same size (not just because some values have been clipped, it seems LePHARE always outputs non-standard lengths of spectra, so that's just great). In order to store the spectra in a nice way, and because we're only interested in a certain wavelength range anyway, we choose a standardised set of wavelength values between 0.4$\mu$m to 5$\mu$m and interpolate the LePHARE spectra.

In [ ]:
num_samples = 1000
wavelength_samples = np.linspace(4e3, 5e4, num_samples)
spectrum_array = np.zeros((len(ids), 3, num_samples)) # ID x Spectrum Type (G/Q/S) x wavelength

for i, idd in tqdm(enumerate(ids), total=len(ids)):
    for obj_num in range(3):
        wavs, flxs = get_lephare_spectrum(idd,obj_num)
        flux_samples = np.interp(wavelength_samples,
                                 wavs, flxs)
        spectrum_array[i,obj_num,:] = flux_samples

In [ ]:
with open('lephare_output.out','r') as f:
    lephare_output_text = f.read()
lephare_df = pd.DataFrame([line.split() for line in lephare_output_text.split('\n')[55:]],
                          columns = ['COADD_OBJECT_ID',
                                     'Z_BEST',
                                     'Z_BEST68_LOW',
                                     'Z_BEST68_HIGH',
                                     'Z_ML',
                                     'CHI_BEST',
                                     'MOD_BEST',
                                     'MAG_ABS_BEST',
                                     'PDZ_BEST',
                                     'SCALE_BEST',
                                     'DIST_MOD_BEST',
                                     'NBAND_USED',
                                     'Z_SEC',
                                     'CHI_SEC',
                                     'MOD_SEC',
                                     'AGE_SEC',
                                     'Z_QSO',
                                     'CHI_QSO',
                                     'MOD_QSO',
                                     'MAG_ABS_QSO',
                                     'DIST_MOD_QSO',
                                     'MOD_STAR',
                                     'CHI_STAR',
                                     'CONTEXT',
                                     'ZSPEC'
                                    ]
                         )[:-1].set_index('COADD_OBJECT_ID')
lephare_df

The above df has 7102 objects, some of which don't have images (there are 6690 objects with 5-band imaging, in `ids`)

In [ ]:
lephare_df_with_imgs = pd.DataFrame(lephare_df.loc[[str(int(str(idd)[-9:])) for idd in ids]].to_numpy(),
                                    columns = lephare_df.columns,
                                    index=ids)
lephare_df_with_imgs.to_csv('lephare_output_data.csv')
lephare_df_with_imgs

In [ ]:
# A 6690x3 array (IDs x G/Q/S)
chi2s = lephare_df_with_imgs.loc[ids][['CHI_BEST','CHI_QSO','CHI_STAR']].to_numpy().astype('float32')

In [ ]:
np.savez_compressed('lephare_spectra_chi2s.npz',
                   ids=ids,
                    wavelengths = wavelength_samples,
                   spectra=spectrum_array,
                   chi2s=chi2s)